In [1]:
import pandas as pd
import json
import os
from datetime import date

# Collecting the data from Twitter

In [2]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])


In [3]:
import pysad

In [99]:
import importlib
importlib.reload(pysad)

<module 'pysad' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad.py'>

In [5]:
init_accounts = pysad.initial_accounts()

In [6]:
init_accounts.list()

['swiss_climate_regular',
 'swiss_climate_controversial',
 'swiss_immigration',
 'french_tech_lesechos']

In [7]:

    
######Choose a category##############    
category_name = 'swiss_climate_controversial'
#category_name = 'swiss_climate_regular'
#category_name = 'french_tech_lesechos'
#category_name = 'swiss_immigration'
#####################################

username_list = init_accounts.accounts(category_name)

# create the path to save the experiment indexed with a date
today = date.today()
date_string = today.strftime("%Y%m%d")
print("date string =", date_string)

#date_string = '191128'

data_path = category_name + '/' + date_string+ '/'
#get_tweets = python_tweets.get_user_timeline(screen_name = username,  
#                                           count = 200, include_rts = True)
if not os.path.isdir(category_name):
    os.mkdir(category_name)
    print('Path created:',category_name)
if not os.path.isdir(data_path):
    os.mkdir(data_path)
    print('Path created:',data_path)
else:
    for f in os.listdir(data_path):
        os.remove(os.path.join(data_path, f))
    print('Cleaned path',data_path)

date string = 20200124
Cleaned path swiss_climate_controversial/20200124/


In [8]:
min_mentions = 2 # minimal number of mentions of a user to be followed
max_day_old = 7 # number max of days in the past
exploration_depth = 2 # mention of mention of mention of ... up to exploration depth

In [9]:
total_user_list = pysad.collect_tweets(username_list, data_path, python_tweets, min_mentions=min_mentions,
               max_day_old=max_day_old, exploration_depth=exploration_depth)

  0%|          | 0/12 [00:00<?, ?it/s]

Threshold set to 2 mentions.

******* Processing users at 0-hop distance *******


  0%|          | 0/26 [00:00<?, ?it/s]

users with empty tweet list or mentions: ['francisrichard', 'templivs']

******* Processing users at 1-hop distance *******


100%|██████████| 26/26 [00:13<00:00,  1.89it/s]

users with empty tweet list or mentions: ['TheOven_lol', 'Actu17']
Total number of users collected:
128 128


# Loading the saved data into an edge table

In [83]:
import glob

edge_df = pd.DataFrame()
for filename in glob.glob(data_path + '*_mentions' +'_t' +str(min_mentions)+ '.json'):
    new_edge_df = pd.read_json(filename)
    print('{} with {} tweets.'.format(filename,len(new_edge_df)))
    edge_df = edge_df.append(new_edge_df)
edge_df.reset_index(drop=True, inplace=True)

swiss_climate_controversial/20200124/RFA_Chinese_mentions_t2.json with 1 tweets.
swiss_climate_controversial/20200124/RestitutorOrien_mentions_t2.json with 42 tweets.
swiss_climate_controversial/20200124/toxisly187_mentions_t2.json with 68 tweets.
swiss_climate_controversial/20200124/AitiDouze_mentions_t2.json with 101 tweets.
swiss_climate_controversial/20200124/cocktail2Funk_mentions_t2.json with 53 tweets.
swiss_climate_controversial/20200124/ClubHorloge_mentions_t2.json with 99 tweets.
swiss_climate_controversial/20200124/QAnonAustria1_mentions_t2.json with 26 tweets.
swiss_climate_controversial/20200124/EnModeMacaron_mentions_t2.json with 6 tweets.
swiss_climate_controversial/20200124/Conflits_FR_mentions_t2.json with 47 tweets.
swiss_climate_controversial/20200124/ITanerof_mentions_t2.json with 78 tweets.
swiss_climate_controversial/20200124/ChalecosAmarill_mentions_t2.json with 116 tweets.
swiss_climate_controversial/20200124/CyrusAParsa1_mentions_t2.json with 6 tweets.
swiss_cl

In [84]:
# display edges with number of hashtags >1
#edge_df[edge_df['hashtags'].apply(lambda x : len(x.split()))>1]
edge_df[edge_df['hashtags'].apply(lambda x : len(x))>1]

,user,mention,weight,hashtags,date,urls,text
45,toxisly187,BasedPoland,1,"[France, Macron]",[2020-01-24 09:22:46],[],"[Massive, absolutely massive crowds out on the..."
47,toxisly187,BreizhOfficiel,2,"[coronavirus, coronovirus]","[2020-01-24 08:34:56, 2020-01-23 15:34:36]",[],"[« J’entends le loup, le renard et la retraite..."
49,toxisly187,ComplotsFaciles,2,"[NousSachons, NousSachons]","[2020-01-24 08:28:32, 2020-01-24 08:28:22]",[],[La version officielle ne tient pas debout. #N...
62,toxisly187,MarionLpz,1,"[Chefing, startupnation]",[2020-01-23 07:38:10],[],"[👉🏼Chez #Chefing, c’est « No asshole policy » ..."
63,toxisly187,Maskloff,1,"[Procureur, Macron]",[2020-01-21 23:01:29],[],[Le #Procureur de Lyon qui a classé le dossier...
...,...,...,...,...,...,...,...
1526,Chabadalala,le_Parisien,2,"[Kaamelott, Rediff]","[2020-01-22 17:20:11, 2020-01-21 00:43:05]",[http://www.leparisien.fr/culture-loisirs/kaam...,[Alexandre Astier dévoile les premières images...
1527,Chabadalala,lpl_zen,1,"[twitter, censure]",[2020-01-21 08:33:30],[],[@lpl_zen le compte @lpl_direct est suspendu.....
1530,Chabadalala,mselon3,1,"[Kouchner, Matzneff]",[2020-01-19 23:48:17],[],"[#Kouchner, signataire d'une pétition pro-pédo..."
1536,Chabadalala,sputnik_fr,1,"[Urgent, Français, Québec, hélicoptère]",[2020-01-22 21:32:36],[],[⚡ #Urgent | #Français disparus au #Québec: un...


In [85]:
edge_df_str = edge_df.copy()
for col in edge_df.columns:
    if isinstance(edge_df[col][0],list) or isinstance(edge_df[col][0],dict):
        edge_df_str[col] = edge_df[col].apply(json.dumps)
        print('Field {} of class {} converted to json string'.format(col,type(edge_df[col][0])))
    else:
        print(col,type(edge_df[col][0]))

user <class 'str'>
mention <class 'str'>
weight <class 'numpy.int64'>
Field hashtags of class <class 'list'> converted to json string
Field date of class <class 'list'> converted to json string
Field urls of class <class 'list'> converted to json string
Field text of class <class 'list'> converted to json string


In [86]:
DEGREE_MIN = 2 # Minimal number of connections in the graph

G = pysad.graph_from_edgeslist(edge_df_str,DEGREE_MIN)
G.name = category_name
G.date = date_string

Creating the graph from the edge list
Nb of nodes: 1312
Nb of nodes after removing nodes with degree strictly smaller than 2: 171
removed 1 isolated nodes.


## Community detection and save full graph

In [87]:
G,clusters = pysad.detect_communities(G)

Communities saved on the graph as node attributes.
Nb of partitions: 7


In [88]:
# Save the graph
import networkx as nx


graphname = '00graph'

graphfilename = data_path + graphname + '_t' + str(min_mentions) + '_md' + str(DEGREE_MIN) +'_graph.gexf'
nx.write_gexf(G,graphfilename)
#nx.write_yaml(G,graphfilename)
#nx.write_pajek(G,graphfilename)
#nx.node_link_data(G,graphfilename)
print('Wrote',graphfilename)

Wrote swiss_climate_controversial/20200124/00graph_t2_md2_graph.gexf


In [89]:
G.date

'20200124'

## Analyze clusters

In [140]:
import importlib
importlib.reload(pysad)

<module 'pysad' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad.py'>

In [126]:
cluster_dfi = pysad.indicator_table(clusters)
cluster_dfi

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2
0,0,19,3,4,0.210526,1.368421,3.210526,4.393352,14.250000,0.473684
1,1,34,3,5,0.147059,1.382353,3.029412,4.187716,20.400000,0.264706
2,2,32,2,12,0.375000,1.250000,1.906250,2.382812,5.333333,0.125000
3,3,24,2,11,0.458333,1.333333,4.583333,6.111111,4.363636,0.166667
4,4,22,2,12,0.545455,1.409091,5.181818,7.301653,3.666667,0.181818
5,5,37,2,16,0.432432,1.324324,2.081081,2.756026,4.625000,0.108108
6,6,2,1,2,1.000000,0.500000,1.500000,0.750000,1.000000,0.500000


In [49]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [127]:
cluster_dfi.sort_values(by='activism',ascending=False)

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2
4,4,22,2,12,0.545455,1.409091,5.181818,7.301653,3.666667,0.181818
3,3,24,2,11,0.458333,1.333333,4.583333,6.111111,4.363636,0.166667
0,0,19,3,4,0.210526,1.368421,3.210526,4.393352,14.250000,0.473684
1,1,34,3,5,0.147059,1.382353,3.029412,4.187716,20.400000,0.264706
5,5,37,2,16,0.432432,1.324324,2.081081,2.756026,4.625000,0.108108
2,2,32,2,12,0.375000,1.250000,1.906250,2.382812,5.333333,0.125000
6,6,2,1,2,1.000000,0.500000,1.500000,0.750000,1.000000,0.500000


In [128]:
cluster_dfi.sort_values(by='in_diversity',ascending=False)

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2
4,4,22,2,12,0.545455,1.409091,5.181818,7.301653,3.666667,0.181818
1,1,34,3,5,0.147059,1.382353,3.029412,4.187716,20.400000,0.264706
0,0,19,3,4,0.210526,1.368421,3.210526,4.393352,14.250000,0.473684
3,3,24,2,11,0.458333,1.333333,4.583333,6.111111,4.363636,0.166667
5,5,37,2,16,0.432432,1.324324,2.081081,2.756026,4.625000,0.108108
2,2,32,2,12,0.375000,1.250000,1.906250,2.382812,5.333333,0.125000
6,6,2,1,2,1.000000,0.500000,1.500000,0.750000,1.000000,0.500000


# Hashtags, dates and urls
Hashtags, dates and urls are on the edges of the network.
We can get the most common hashtags within a community and also betwenn communities using the edges that connect them.

In [132]:
cluster_df = pysad.dates_tags_table(clusters)
cluster_df.iloc[:,:-2]

,Community,Average date,Deviation (days),hashtag0,hashtag1,hashtag2,hashtag3,hashtag4
0,0,2020-01-23,0,Chine,coronavirus,Wuhan,virus,Coronavirus
1,1,2020-01-22,1,GiletsJaunes,reformedesretraites,greve24janvier,greve23janvier,CGT
2,2,2020-01-22,1,France,Macron,Retraites,Blackrock,Migrant
3,3,2020-01-23,1,AI,Google,balancetataupe,5G,Facebook
4,4,2020-01-24,0,HerveRyssen,réémigration,Immigration,DissidenceFrançaise,LesFrançaisDabord
5,5,2020-01-21,1,Macron,Paris,Benalla,Immigration,Kosovo
6,6,2020-01-19,1,,,,,


In [134]:
community_table = cluster_dfi.merge(cluster_df)
community_table = community_table.iloc[:,:-2]
community_table

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2,Average date,Deviation (days),hashtag0,hashtag1,hashtag2,hashtag3,hashtag4
0,0,19,3,4,0.210526,1.368421,3.210526,4.393352,14.250000,0.473684,2020-01-23,0,Chine,coronavirus,Wuhan,virus,Coronavirus
1,1,34,3,5,0.147059,1.382353,3.029412,4.187716,20.400000,0.264706,2020-01-22,1,GiletsJaunes,reformedesretraites,greve24janvier,greve23janvier,CGT
2,2,32,2,12,0.375000,1.250000,1.906250,2.382812,5.333333,0.125000,2020-01-22,1,France,Macron,Retraites,Blackrock,Migrant
3,3,24,2,11,0.458333,1.333333,4.583333,6.111111,4.363636,0.166667,2020-01-23,1,AI,Google,balancetataupe,5G,Facebook
4,4,22,2,12,0.545455,1.409091,5.181818,7.301653,3.666667,0.181818,2020-01-24,0,HerveRyssen,réémigration,Immigration,DissidenceFrançaise,LesFrançaisDabord
5,5,37,2,16,0.432432,1.324324,2.081081,2.756026,4.625000,0.108108,2020-01-21,1,Macron,Paris,Benalla,Immigration,Kosovo
6,6,2,1,2,1.000000,0.500000,1.500000,0.750000,1.000000,0.500000,2020-01-19,1,,,,,


In [135]:
community_table.sort_values('activism',ascending=False).drop(['Average date','Deviation (days)'],axis=1)

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2,hashtag0,hashtag1,hashtag2,hashtag3,hashtag4
4,4,22,2,12,0.545455,1.409091,5.181818,7.301653,3.666667,0.181818,HerveRyssen,réémigration,Immigration,DissidenceFrançaise,LesFrançaisDabord
3,3,24,2,11,0.458333,1.333333,4.583333,6.111111,4.363636,0.166667,AI,Google,balancetataupe,5G,Facebook
0,0,19,3,4,0.210526,1.368421,3.210526,4.393352,14.250000,0.473684,Chine,coronavirus,Wuhan,virus,Coronavirus
1,1,34,3,5,0.147059,1.382353,3.029412,4.187716,20.400000,0.264706,GiletsJaunes,reformedesretraites,greve24janvier,greve23janvier,CGT
5,5,37,2,16,0.432432,1.324324,2.081081,2.756026,4.625000,0.108108,Macron,Paris,Benalla,Immigration,Kosovo
2,2,32,2,12,0.375000,1.250000,1.906250,2.382812,5.333333,0.125000,France,Macron,Retraites,Blackrock,Migrant
6,6,2,1,2,1.000000,0.500000,1.500000,0.750000,1.000000,0.500000,,,,,


In [136]:
community_table[['Community', 'nb_nodes', 'activism','hierarchy', 'hashtag0', 'hashtag1', 'hashtag2', 'hashtag3', 'hashtag4']].sort_values('activism',ascending=False)

,Community,nb_nodes,activism,hierarchy,hashtag0,hashtag1,hashtag2,hashtag3,hashtag4
4,4,22,7.301653,3.666667,HerveRyssen,réémigration,Immigration,DissidenceFrançaise,LesFrançaisDabord
3,3,24,6.111111,4.363636,AI,Google,balancetataupe,5G,Facebook
0,0,19,4.393352,14.250000,Chine,coronavirus,Wuhan,virus,Coronavirus
1,1,34,4.187716,20.400000,GiletsJaunes,reformedesretraites,greve24janvier,greve23janvier,CGT
5,5,37,2.756026,4.625000,Macron,Paris,Benalla,Immigration,Kosovo
2,2,32,2.382812,5.333333,France,Macron,Retraites,Blackrock,Migrant
6,6,2,0.750000,1.000000,,,,,


In [139]:
cluster_df[['Community','urls']]

,Community,urls
0,0,"[https://trib.al/tKd7vuc, https://trib.al/tKd7..."
1,1,[https://twitter.com/AiphanMarcel/status/12188...
2,2,[https://www.nouvelordremondial.cc/2020/01/22/...
3,3,[https://twitter.com/AldoSterone111/status/121...
4,4,[https://www.monde-diplomatique.fr/2008/08/SAN...
5,5,[https://actu17.fr/la-rochelle-un-homme-court-...
6,6,[https://www.youtube.com/watch?v=9j-OwBe7XPE&f...


### Process the urls

In [143]:
url_table = pysad.get_urls(cluster_df[['Community','urls']])
url_table = pysad.convert_bitly(url_table)
filtered_url_table = pysad.drop_twitter_urls(url_table)

In [145]:
filtered_url_table

,url,Community,Occurence
0,https://www.businessbourse.com/2020/01/23/fran...,1,1
1,http://www.fdesouche.com/1324803-pays-bas-une-...,5,1
2,http://www.leparisien.fr/amp/politique/lrem-le...,1,1
3,http://www.leparisien.fr/culture-loisirs/kaame...,5,1
4,https://actu17.fr/la-rochelle-un-homme-court-v...,5,1
5,https://home.solari.com/mind-control-tactics-u...,3,1
6,https://jovanovic.com/blog.htm#consequence-du-,2,1
7,https://jovanovic.com/blog.htm#le-systeme-banc,2,2
8,https://l.leparisien.fr/0RN-6,5,1
9,https://trib.al/tKd7vuc,0,2


# Sort users by community and save in a excel sheet

In [ ]:
# Sort users by community and store their node degree (importance)
community_nodes = {}
for node,data in G.nodes(data=True):
    community_nb = data['community']
    if  community_nb not in community_nodes:
        community_nodes[community_nb] = [(node, G.degree(node))]
    else:
        community_nodes[community_nb].append((node, G.degree(node)))


# Display the exmaple of community c_idx
#c_idx = 0
#ddf = pd.DataFrame(community_nodes[c_idx],columns=['User','Degree'])
#print('list of most connected users in community',c_idx)
#ddf.sort_values(by='Degree',ascending=False).head(20)

In [ ]:
# Save to an excel file
with pd.ExcelWriter(data_path + 'graph_infos.xlsx') as writer:
    for community_nb in community_nodes:
        ddf = pd.DataFrame(community_nodes[community_nb],columns=['User','Degree'])
        ddf = ddf.sort_values(by='Degree',ascending=False)#.head(20)
        ddf.to_excel(writer, sheet_name='Community_' + str(community_nb),index=False)
    community_table.to_excel(writer, sheet_name='Hashtags',index=False)
    #users_df.to_excel(writer, sheet_name='Initial_users_details',index=False)
    filtered_url_table.to_excel(writer, sheet_name='List_of_urls',index=False)
    # Set the column width
    column_width = 25
    for sheet in writer.sheets: 
        worksheet = writer.sheets[sheet]
        for col in ['A','B','C','D','E','F','G','H']:
            worksheet.column_dimensions[col].width = column_width
    writer.sheets['List_of_urls'].column_dimensions['A'].width = 100